In [7]:
import cv2
import time
import mediapipe as mp
import numpy as np

In [8]:
CAPTURE_DELAY = 1.0                 # seconds a digit must stay stable
OPS           = ['+', '-', '*', '/']
WINDOW_TITLE  = "Air-Calc (auto-capture)"

In [9]:
#Mediapipe setup
mp_hands = mp.solutions.hands
hands    = mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.7)
draw     = mp.solutions.drawing_utils
style    = mp.solutions.drawing_styles.get_default_hand_landmarks_style()

In [10]:
# Finger‑count helper (0‑5 per hand)
def fingers_up(landmarks, handedness):
    """Return number of raised fingers (0-5) for one hand."""
    # Thumb rule differs for left/right hand (x-coordinate check)
    tips  = [4, 8, 12, 16, 20]
    mcp   = [2, 6, 10, 14, 18]       # joints below each tip
    count = 0
    thumb_tip_x = landmarks[tips[0]].x
    thumb_mcp_x = landmarks[mcp[0]].x
    for i in range(1, 5):            # four fingers
        if landmarks[tips[i]].y < landmarks[mcp[i]].y:
            count += 1
    # thumb
    if handedness == 'Right' and thumb_tip_x > thumb_mcp_x:
        count += 1
    if handedness == 'Left'  and thumb_tip_x < thumb_mcp_x:
        count += 1
    return count

In [11]:
def calc(a, b, op):
    """Simple calculator."""
    return {'+': a+b, '-': a-b, '*': a*b, '/': a/b if b else np.inf}[op]


In [12]:


def draw_overlay(frame, state):
    """Render UI text on the frame."""
    y, dy = 25, 25
    cv2.putText(frame, f"Operator: {OPS[state['op_idx']]} (o to cycle)",
                (10, y), 0, .8, (0,255,255), 2)

    cv2.putText(frame,
        "Hold digit 2s to capture • b=start B • n=no B • a=action",
        (10, y+dy), 0, .55, (200,200,0), 1)

    cv2.putText(frame, f"A: {state['A_str'] or '-'}",
                (10, y+2*dy), 0, .9,
                (0,255,0) if state['current']=='A' else (180,180,180), 2)
    cv2.putText(frame, f"B: {state['B_str'] or '-'}",
                (10, y+3*dy), 0, .9,
                (0,255,0) if state['current']=='B' else (180,180,180), 2)

    if state['digit'] is not None:
        cv2.putText(frame, f"Detected: {state['digit']}",
                    (10, y+4*dy), 0, .8, (255,120,0), 2)
    if state['stable_since']:
        rem = CAPTURE_DELAY - (time.time() - state['stable_since'])
        cv2.putText(frame, f"Capture in: {rem:.1f}s",
                    (220, y+4*dy), 0, .7, (150,200,150), 2)
    if state['result']:
        cv2.putText(frame, state['result'],
                    (10, y+5*dy), 0, 1.0, (0,200,255), 3)

def update_digit_state(state, digit):
    """Handle stability timer & auto-capture."""
    now = time.time()
    if digit != state['prev_digit']:
        state['prev_digit']   = digit
        state['stable_since'] = now if digit is not None else None
    elif digit is not None and state['stable_since'] \
            and (now - state['stable_since']) >= CAPTURE_DELAY:
        # safe to capture
        bank = 'A_digits' if state['current'] == 'A' else 'B_digits'
        if len(state[bank]) < 3:
            state[bank].append(digit)
        # reset
        state['prev_digit'] = None
        state['stable_since'] = None

def key_controls(key, state):
    """React to keyboard input."""
    if key == ord('o'):                             # cycle operator
        state['op_idx'] = (state['op_idx'] + 1) % 4
    elif key == ord('r'):                           # reset
        state.update(A_digits=[], B_digits=[], result='')
        state['current'] = 'A'
    elif key == ord('b'):                           # switch to B
        state['current'] = 'B'
    elif key == ord('n'):                           # unary: A only
        if state['A_digits']:
            a = int(''.join(map(str, state['A_digits'])))
            state['result'] = f"Result = {a} (no B)"
    elif key == ord('a'):                           # compute A (op) B
        if state['A_digits']:
            a = int(''.join(map(str, state['A_digits'])))
            b = int(''.join(map(str, state['B_digits']))) if state['B_digits'] else 0
            op = OPS[state['op_idx']]
            state['result'] = f"{a} {op} {b} = {calc(a, b, op)}"

# ────────── Main loop ──────────
state = dict(
    op_idx=0,        A_digits=[], B_digits=[],
    prev_digit=None, stable_since=None, current='A', result=''
)

cap = cv2.VideoCapture(0)
cv2.namedWindow(WINDOW_TITLE, cv2.WINDOW_NORMAL)

while cap.isOpened():
    ok, frame = cap.read()
    if not ok:
        break

    frame = cv2.flip(frame, 1)
    rgb   = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    res   = hands.process(rgb)

    # ── Detect digit across both hands ──
    digit = None
    if res.multi_hand_landmarks and len(res.multi_hand_landmarks) == 2:
        counts = []
        for hlm, hness in zip(res.multi_hand_landmarks,
                              res.multi_handedness):
            counts.append(fingers_up(hlm.landmark,
                                     hness.classification[0].label))
            draw.draw_landmarks(frame, hlm, mp_hands.HAND_CONNECTIONS, style)
        total = sum(counts)
        if 0 <= total <= 9:
            digit = total
    state['digit'] = digit

    # ── Stabilize & maybe capture ──
    update_digit_state(state, digit)

    # ── Prepare strings for overlay ──
    state['A_str'] = ''.join(map(str, state['A_digits']))
    state['B_str'] = ''.join(map(str, state['B_digits']))

    # ── UI & Window ──
    draw_overlay(frame, state)
    cv2.imshow(WINDOW_TITLE, frame)

    # ── Key events ──
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    key_controls(key, state)

cap.release()
cv2.destroyAllWindows()
